In [ ]:
!pip install keras==2.2.4
!pip install -q pydot
!pip install graphviz
!apt-get install graphviz
from keras.utils.vis_utils import plot_model
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install tensorflow==1.15.0
import os
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from gensim.models import Word2Vec
from gensim import models
from gensim.models.keyedvectors import KeyedVectors
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
import csv
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from tqdm import tqdm, trange
from google.colab import files
uploaded = files.upload()

data_raw = pd.read_csv("data.csv")

print("Number of rows in data =",data_raw.shape[0])
print("Number of columns in data =",data_raw.shape[1])
print("\n")
print("**Sample data:**")

data_raw.head()

In [ ]:
#One hot encoding for all labels
data_raw
j=0
for i in data_raw.iterrows():
    if data_raw.worry[j]>=5:
        data_raw.worry[j]=1
    else:
        data_raw.worry[j]=0
    j=j+1
j=0
for i in data_raw.iterrows():
    if data_raw.anger[j]>=5:
        data_raw.anger[j]=1
    else:
        data_raw.anger[j]=0
    j=j+1
j=0
for i in data_raw.iterrows():
    if data_raw.anxiety[j]>=5:
        data_raw.anxiety[j]=1
    else:
        data_raw.anxiety[j]=0
    j=j+1
j=0
for i in data_raw.iterrows():
    if data_raw.sadness[j]>=5:
        data_raw.sadness[j]=1
    else:
        data_raw.sadness[j]=0
    j=j+1
j=0
for i in data_raw.iterrows():
    if data_raw.desire[j]>=5:
        data_raw.desire[j]=1
    else:
        data_raw.desire[j]=0
    j=j+1

j=0
for i in data_raw.iterrows():
    if data_raw.disgust[j]>=5:
        data_raw.disgust[j]=1
    else:
        data_raw.disgust[j]=0
    j=j+1
j=0
for i in data_raw.iterrows():
    if data_raw.fear[j]>=5:
        data_raw.fear[j]=1
    else:
        data_raw.fear[j]=0
    j=j+1
j=0
for i in data_raw.iterrows():
    if data_raw.happiness[j]>=5:
        data_raw.happiness[j]=1
    else:
        data_raw.happiness[j]=0
    j=j+1
j=0
for i in data_raw.iterrows():
    if data_raw.relaxation[j]>=5:
        data_raw.relaxation[j]=1
    else:
        data_raw.relaxation[j]=0
    j=j+1


        

data_raw


In [ ]:

MAX_SEQUENCE_LENGTH=50
data_raw.shape


In [ ]:
emotion_labels=data_raw[['worry','anger','disgust','fear','anxiety','sadness','happiness','relaxation','desire']]
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size
plt.xlabel('Emotions')
plt.ylabel('Total number of tweets')
plt.title('Distribution of tweets in India at a glance')

emotion_labels.sum(axis=0).plot.bar()


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
def preprocess_text(sen): 
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence
data_raw['Text_Clean'] = data_raw['text_long'].apply(lambda x: preprocess_text(x))


import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize, WordNetLemmatizer


tokens = [word_tokenize(sen) for sen in data_raw.Text_Clean]
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 


from nltk.corpus import stopwords
stoplist = stopwords.words('english')

def remove_stop_words(tokens):
    return [word for word in tokens if word not in stoplist]

filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 
result = [' '.join(sen) for sen in filtered_words]
data_raw['Text_Final'] = result
data_raw['tokens'] = filtered_words
data_raw=data_raw[['Text_Final', 'tokens','worry','anger','disgust','fear','anxiety','sadness','happiness','relaxation','desire']]

In [ ]:
data_raw

In [ ]:
!pip install -U sentence-transformers
sentences= data_raw.Text_Final.values.tolist()
from sentence_transformers import SentenceTransformer
vec = SentenceTransformer('bert-base-nli-mean-tokens')


In [ ]:
data_train, data_test = train_test_split(data_raw, test_size=0.10, random_state=42)
data_raw['Text_Final']

In [ ]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

In [ ]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

In [ ]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 768

In [ ]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))


In [ ]:
num_words = len(train_word_index)
train_rnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
len(training_sequences)


In [ ]:
train_word_index

In [ ]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    
    train_embedding_weights[index,:] = vec.encode(word)
print(train_embedding_weights.shape)


In [ ]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_rnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
data_test.shape